In [8]:
import numpy as np
import pandas as pd
import requests
import os
from PyPDF2 import PdfFileMerger
import PyPDF2

In [2]:
csvfile=pd.read_csv('2019-resume-submission-form-career-opportunities-registration-2019-02-21.csv')

In [3]:
columnReplace={
    'Name (First)':'First Name',
'Name (Middle Initial)':'Middle Initial',
'Name (Last)':"Last Name",
'Address (State / Province)':'State / Province',
'Address (Country)':'Country'
}
csvfile=csvfile.rename(columnReplace, axis=1)

Group={}
notorganizer=['Olivo','Georgatos','Bradley','Overstreet','Suder','Flores']
isOrganizer=[]
for idx, row in csvfile.iterrows():
    if ('mit' in str(row['Current Company or School']).lower()) and (row['Student / Professional Status']=='Grad Student') and (row['Last Name'] not in str(notorganizer)):
        isOrganizer.append(1)
    else:
        isOrganizer.append(0)

csvfile['isOrganizer']=isOrganizer




#csvfile=csvfile.drop(['Payment Amount','Payment Status'],axis=1)
colname=csvfile.columns
colname=[x[:-2]+'(NE)' if x[-2:]=='.1' else x for x in colname]
csvfile.columns=colname
csvfile.groupby('Student / Professional Status').count()
                                                    
Group['CO']=csvfile[np.logical_and(csvfile['Student / Professional Status']=='Grad Student', csvfile['isOrganizer']==1)]
Group['US']=csvfile[csvfile['Student / Professional Status']=='College Student']
Group['GS']=csvfile[np.logical_and(csvfile['Student / Professional Status']=='Grad Student', csvfile['isOrganizer']==0)]
Group['HS']=csvfile[csvfile['Student / Professional Status']=='High School Student']
Group['CP']=csvfile[csvfile['Student / Professional Status']=='In the Workforce']

In [4]:
sum(isOrganizer)

16

In [5]:
row['Last Name']

'Wade'

In [6]:
dfList=[]
blackList=['ankurverma','shaileeshah','spencerhook','danielbyrne','danielworth','yifeihu']
for key, gp in Group.items():
    #if key in ['US','HS','CP']:
        #continue
    print('Category {}'.format(key))
    if not os.path.isdir('..\Resume Book\{}'.format(key)):
        os.mkdir('..\Resume Book\{}'.format(key))
    gp=gp.drop_duplicates(subset=['First Name','Last Name','Email Address'])    
    gp['lowercase name']=[(x+y).lower() for x,y in zip(gp['First Name'],gp['Last Name'])]
    print(gp.shape)
    gp=gp[gp['lowercase name'].apply(lambda x: True if x not in blackList else False)]
    print(gp.shape)
    gp=gp.sort_values(['lowercase name'])
    gp['ResumeBookId']=np.array(range(1,len(gp)+1))
    gp['Group']=key
    """
    for idx, row in gp.iterrows():
        url = row['Resume Submission']
        name= row['First Name']+row['Last Name']+str(row['ResumeBookId'])
        r = requests.get(url, allow_redirects=True)
        open('..\Resume Book\{}\{}.pdf'.format(key,name), 'wb').write(r.content)
    """
    dfList.append(gp)

Category CO
(16, 42)
(16, 42)
Category US
(181, 42)
(179, 42)
Category GS
(252, 42)
(251, 42)
Category HS
(9, 42)
(9, 42)
Category CP
(88, 42)
(85, 42)


C:\Users\huang\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
###Merging

def merger_pdf(pdf_dir,key):
    merger = PdfFileMerger()
    file_list = os.listdir(pdf_dir)
    flag=0
    count=0
    for file_name in file_list:
        #print(pdf_dir + file_name)      
        
        """if 'Ankur' in file_name:
            flag=1
            continue
        if flag==0:
            continue
        if 'YifeiHu' in file_name:
            break"""
        input_pdf = open(pdf_dir+file_name, "rb")
        merger.append(input_pdf)
        count=count+1
    outputfilename='..\\Resume Book\\'+key+"output.pdf"
    output = open(outputfilename, "wb")
    merger.write(output)
    print('{} pdf merged'.format(count))
    #merger.encrypt('password')
    
                 
for key, gp in Group.items():
    if key not in ['CO']:
        continue
    print('Current Category: {}'.format(key))
    path='..\\Resume Book\\{}\\'.format(key)
    merger_pdf(path,key)

Current Category: CO


18 pdf merged


In [11]:
def addpassword(pdf_dir,key): 
    outputfilename='..\\Resume Book\\'+key+"output.pdf"
    pdfFile = open(outputfilename, 'rb')
    # Create reader and writer object
    pdfReader = PyPDF2.PdfFileReader(pdfFile)
    pdfWriter = PyPDF2.PdfFileWriter()
    # Add all pages to writer (accepted answer results into blank pages)
    count=0
    for pageNum in range(pdfReader.numPages):
        pdfWriter.addPage(pdfReader.getPage(pageNum))
        count=count+1
    # Encrypt with your password
    pdfWriter.encrypt('2019sLoAn')
    # Write it to an output file. (you can delete unencrypted version now)
    resultPdf = open('..\\Resume Book\\'+key+"output_encrypted.pdf", 'wb')
    print('resultPdf has {} pages'.format(count))
    pdfWriter.write(resultPdf)
    resultPdf.close()
    
for key, gp in Group.items():
    if key not in ['HS']:
        continue
    print('Current Category: {}'.format(key))
    path='..\\Resume Book\\{}\\'.format(key)
    addpassword(path,key)

Current Category: HS
resultPdf has 11 pages


In [13]:
merged=pd.concat(dfList)
merged=merged.drop(['Resume Submission','isOrganizer','lowercase name'],axis=1)
merged['Resume Book ID']=[y+'.'+str(x).zfill(3) for x,y in zip(merged['ResumeBookId'],merged['Group'])]
merged=merged.drop(['ResumeBookId','Group'],axis=1)

In [14]:
merged.to_excel('ResumeBookDirectory-2019Jan21.xls')